In [ ]:
!pip -q install -U timm==1.0.9 scikit-learn==1.5.2 pillow==10.4.0 opencv-python==4.10.0.84
!pip -q install --index-url https://download.pytorch.org/whl/cu121 torch==2.4.0 torchvision==0.19.0

import torch, platform
print('torch =', torch.__version__)
if torch.cuda.is_available():
    print('cuda  =', torch.cuda.get_device_name(0))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 10.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.5 MB/s eta 0:00:00
     

In [ ]:
!unzip -q /content/drive/MyDrive/виды.zip -d /content/

In [ ]:
from pathlib import Path
import random, shutil

src = Path('/content/виды')
dst = Path('/content/species_splits')
val_ratio = 0.15
dst.mkdir(parents=True, exist_ok=True)

for cls in sorted(p for p in src.iterdir() if p.is_dir()):
    imgs = [p for p in cls.rglob('*') if p.suffix.lower() in {'.jpg','.jpeg','.png','.bmp','.webp'}]
    random.shuffle(imgs)
    n_val = max(1, int(len(imgs)*val_ratio))
    for split, subset in [('val', imgs[:n_val]), ('train', imgs[n_val:])]:
        out = (dst/split/cls.name); out.mkdir(parents=True, exist_ok=True)
        for p in subset:
            shutil.copy2(p, out/p.name)
print('train/val готово:', (dst/'train').exists(), (dst/'val').exists())

train/val готово: True True


In [ ]:
import torch, os, platform
print("torch:", torch.__version__)
print("torch.cuda.is_available:", torch.cuda.is_available())
print("torch.version.cuda (build):", torch.version.cuda)
if torch.cuda.is_available():
    print("device:", torch.cuda.get_device_name(0))
!nvidia-smi

torch: 2.4.0+cu121
torch.cuda.is_available: True
torch.version.cuda (build): 12.1
device: Tesla T4
Thu Oct 30 13:48:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   74C    P8             23W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                        

In [ ]:
import torch, time
torch.set_float32_matmul_precision('high')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device =', device)

# прогрев
x = torch.randn(8192, 8192, device=device)
for _ in range(3):
    y = x @ x
torch.cuda.synchronize()

# замер
t0 = time.time()
for _ in range(5):
    y = x @ x
torch.cuda.synchronize()
print('5 GEMM ops took:', time.time()-t0, 'sec')

device = cuda
5 GEMM ops took: 1.4950735569000244 sec


In [ ]:
!python /content/train_species_colab.py \
  --data /content/species_splits \
  --out  /content/runs/species_convnext_tiny_main \
  --model convnextv2_tiny.fcmae_ft_in1k \
  --img 384 --bs 32 --epochs 60 \
  --num-workers 4 --compile \
  --mixup 0.2 --cutmix 0.2 --label-smoothing 0.1

Using device: cuda | torch=2.4.0+cu121 | timm=1.0.9
CUDA device: Tesla T4
Classes: 19 | train=2174 val=375 | counts=[89, 47, 104, 163, 65, 161, 278, 126, 95, 91, 117, 275, 52, 105, 39, 141, 24, 120, 82]
Model compiled with torch.compile
/content/train_species_colab.py:158: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device == 'cuda'))
Epoch 1/60:   0%|                                                            | 0/68 [00:00<?, ?it/s]/content/train_species_colab.py:179: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device == 'cuda')):
Epoch 1/60:   1%|▎                       | 1/68 [01:06<1:14:40, 66.87s/it, loss=3.0107, lr=6.00e-05]/content/train_species_colab.py:179: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `

In [ ]:
!python /content/export_species_colab.py \
  --ckpt /content/runs/species_convnext_tiny_main/best.pt \
  --out  /content/exports \
  --model convnextv2_tiny.fcmae_ft_in1k \
  --img 384

/content/export_species_colab.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(args.ckpt, map_location='cpu')  # keep weights_only=False, we need metad

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!ls -lah /content/exports/species

TARGET="/content/drive/MyDrive/exports/species"
!mkdir -p "$TARGET"

!rsync -ah --info=progress2 /content/exports/species/ "$TARGET"/

Mounted at /content/drive
total 107M
drwxr-xr-x 2 root root 4.0K Oct 30 15:02 .
drwxr-xr-x 3 root root 4.0K Oct 30 14:57 ..
-rw-r--r-- 1 root root 107M Oct 30 15:02 model_ts.pt
-rw-r--r-- 1 root root  398 Oct 30 15:02 species_classes.json
-rw-r--r-- 1 root root  756 Oct 30 15:02 species_ru_map.json
        111.91M 100%  237.62MB/s    0:00:00 (xfr#3, to-chk=0/4)
